In [ ]:
import sys
sys.version

In [ ]:
import tqdm
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

In [ ]:
image_paths, structure_path = rai.download_deepmind_example()
structure_path

In [ ]:
grids, image_stack, image_uids = rai.paths_to_image_stack_hfs(cfg=cfg, paths=image_paths)

In [ ]:
models = rai.load_models(cfg=cfg)

In [ ]:
predicted_masks = rai.inference(
    cfg=cfg,
    models=models,
    image_stack=image_stack,
    max_batch_size=10,
    step_size=40,
)

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
align_map = {
    "Brain": [TG263.Brain],
    "Brainstem": [TG263.Brainstem],
    "Cochlea-Lt": [TG263.Cochlea_L],
    "Cochlea-Rt": [TG263.Cochlea_R],
    "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
    "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
    "Lens-Lt": [TG263.Lens_L],
    "Lens-Rt": [TG263.Lens_R],
    "Lung-Lt": [TG263.Lung_L],
    "Lung-Rt": [TG263.Lung_R],
    "Mandible": [TG263.Bone_Mandible],
    "Optic-Nerve-Lt": [TG263.OpticNrv_L],
    "Optic-Nerve-Rt": [TG263.OpticNrv_R],
    "Orbit-Lt": [TG263.Eye_L],
    "Orbit-Rt": [TG263.Eye_R],
    "Parotid-Lt": [TG263.Parotid_L],
    "Parotid-Rt": [TG263.Parotid_R],
    "Spinal-Cord": [TG263.SpinalCord],
    "Submandibular-Lt": [TG263.Glnd_Submand_L],
    "Submandibular-Rt": [TG263.Glnd_Submand_R],
}
structure_names = list(align_map.keys())

dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=structure_names
)



In [ ]:
predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    x_grid=x_grid, y_grid=y_grid, masks=predicted_masks, structure_names=cfg["structures"]
)

aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
    predicted_contours_by_structure, align_map
)

dice = {}
for name in align_map:
    dice[name] = rai.dice_from_contours_by_slice(
        dicom_contours_by_structure[name],
        aligned_predicted_contours_by_structure[name],
    )
    
dice

In [ ]:
# Step-size: 50

# {'Brain': 0.9770559873907146,
#  'Brainstem': 0.7446594757682403,
#  'Cochlea-Lt': 0.0,
#  'Cochlea-Rt': 0.0,
#  'Lacrimal-Lt': 0.4287916964134596,
#  'Lacrimal-Rt': 0.7150097289790804,
#  'Lens-Lt': 0.6240236186258522,
#  'Lens-Rt': 0.6694594192976419,
#  'Lung-Lt': 0.987162664054495,
#  'Lung-Rt': 0.9883733539708488,
#  'Mandible': 0.8933116995749084,
#  'Optic-Nerve-Lt': 0.5987591575284228,
#  'Optic-Nerve-Rt': 0.7122899293396878,
#  'Orbit-Lt': 0.9375315112299282,
#  'Orbit-Rt': 0.9348481957064191,
#  'Parotid-Lt': 0.7269063671274066,
#  'Parotid-Rt': 0.737441485353478,
#  'Spinal-Cord': 0.7909805731621938,
#  'Submandibular-Lt': 0.7509269778519194,
#  'Submandibular-Rt': 0.5433672367481709}

In [ ]:
# grids = (z_grid, y_grid, x_grid)
# slice_indices, axis_limits, centre_indices = rai.view_ranges_from_masks(grids, predicted_masks)

In [ ]:
# contours_by_orientation = rai.masks_to_contours_by_orientation(
#     grids, predicted_masks, structure_names=cfg["structures"]
# )

In [ ]:
# rai.draw_contours_by_orientation(
#     cfg=cfg,
#     grids=grids,
#     images=image_stack,
#     contours_by_orientation=contours_by_orientation,
#     vmin=0.2,
#     vmax=0.4,
#     axis_limits=axis_limits,
#     centre_indices=centre_indices,
# )

In [ ]:
plt.close('all')

# rai.auto_scroll_contours_by_orientation(
#     grids=grids,
#     images=image_stack,
#     contours_by_orientation=contours_by_orientation,
#     slice_indicies=slice_indices,
#     axis_limits=axis_limits,
#     structure_names=cfg["structures"],
#     vmin=0.2,
#     vmax=0.4,
# )

In [ ]:
# TODO:
# * Change figures to clickable interactive transverse/coronal/sagital bokeh
# * Calculate and report hausdorff and surface dice as well